# 02. 장단기 메모리(Long Short-Term Memory, LSTM)

가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN) = 케라스에서는 SimpleRNN

## 1. 바닐라 RNN의 한계
바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있다 -> time step이 깊어질수록 앞의 정보가 뒤로 충분히 전달이 안된다.  
### 이를 장기 의존성 문제(the problem of Long-Term Dependencies)라고 한다.

<img src="https://wikidocs.net/images/page/22888/vanilla_rnn_ver2.PNG" width="300" height="200">  

$h_{t} = tanh(W_{x}x_{t} + W_{h}h_{t−1} + b)$

바닐라 RNN은 $x_{t}$와 $h_{t-1}$이라는 두 개의 입력이 각각의 가중치와 곱해져서 메모리 셀의 입력이 됩니다. 그리고 이를 하이퍼볼릭탄젠트 함수의 입력으로 사용하고 이 값은 은닉층의 출력인 은닉 상태가 됩니다.

## 3. LSTM(Long Short-Term Memory)

<img src="https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG" width="300" height="200">  

전통적인 RNN의 이러한 단점을 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 합니다. LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정합니다. 요약하면 LSTM은 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가하였습니다.
-> 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보인다. 

<img src="https://wikidocs.net/images/page/22888/cellstate.PNG" width="300" height="200">  

셀 상태는 위의 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선입니다. 셀 상태 또한 이전에 배운 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용됩니다.

은닉 상태값과 셀 상태값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용합니다. 각 게이트는 삭제 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재합니다. 시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절합니다

## (1) 입력 게이트
$i_{t}=σ(W_{xi}x_{t}+W_{hi}h_{t-1}+b_{i})$
$g_{t}=tanh(W_{xg}x_{t}+W_{hg}h_{t-1}+b_{g})$
* 입력 게이트는 현재 정보를 기억하기 위한 게이트
* 시그모이드 함수를 지나 0과 1 사이의 값과 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값 두 개가 나오게 된다.  이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정한다. 
## (2) 삭제 게이트
$f_{t}=σ(W_{xf}x_{t}+W_{hf}h_{t-1}+b_{f})$
* 시그모이드 함수를 지나면 0과 1 사이의 값이 나오게 되는데, 이 값이 곧 삭제 과정을 거친 정보의 양입니다. 0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것입니다.
## (3) 셀 상태(장기 상태)
$C_{t}=f_{t}∘C_{t-1}+i_{t}∘g_{t}$ -> LSTM에서는 장기 상태라고 부르기도 합니다
* 입력게이트에서 구한 $i_{t},g_{t}$이 두 개의 값에 대해서 원소별 곱(entrywise product)을 진행
* 입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더합니다. 
* 이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨집니다.
* 결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정합니다.
## (4) 출력 게이트와 은닉 상태(단기 상태)
$o_{t}=σ(W_{xo}x_{t}+W_{ho}h_{t-1}+b_{o})$
$h_{t}=o_{t}∘tanh(c_{t})$
* 출력 게이트는 현재 시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값입니다. 해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰이게 됩니다.
* 은닉 상태 = 단기 상태
* 은닉 상태는 장기 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값
* 해당 값은 출력 게이트의 값과 연산되면서, 값이 걸러지는 효과가 발생합니다. 단기 상태의 값은 또한 출력층으로도 향합니다.

4. 파이토치의 nn.LSTM()

# 4. 파이토치의 nn.LSTM()
nn.LSTM(input_dim, hidden_size, batch_fisrt=True)  